In [1]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime

#add the access token you got from NOAA
Token = 'lwBmDJlPCkqeMyurcljRSVQHVFVhhHDt'

In [2]:
# Get IDs of all stations that have temperature data 
df = pd.DataFrame()

# make the api call
for offset in range(1, 41000, 1000):
    print(offset) 
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datacategoryid=TEMP&offset='+str(offset)+'&limit=1000&datasetid=GHCND', headers={'token':Token})
    json_result = json.loads(r.text)
    df = pd.concat([df, pd.DataFrame(json_result['results'])])

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001


/home/amy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


32001
33001
34001
35001
36001
37001
38001
39001
40001


In [3]:
# make formatting consistent with bit beforehand 
df.id = df.id.replace({'GHCND:':''}, regex=True)

In [4]:
# only keep columns in date range 
select_dates = df[(df['mindate'] <= '2012-1-1') & (df['maxdate'] >= '2017-12-31')]
select_dates

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name
2,0.7282,34.0,METERS,AE000041196,25.333,55.517,2020-02-04,1944-03-20,"SHARJAH INTER. AIRP, AE"
3,0.9971,10.4,METERS,AEM00041194,25.255,55.364,2020-02-04,1983-01-01,"DUBAI INTERNATIONAL, AE"
4,0.9957,26.8,METERS,AEM00041217,24.433,54.651,2020-02-04,1983-01-02,"ABU DHABI INTERNATIONAL, AE"
5,0.9919,264.9,METERS,AEM00041218,24.262,55.609,2020-02-04,1994-03-23,"AL AIN INTERNATIONAL, AE"
7,0.4729,977.2,METERS,AFM00040938,34.210,62.228,2020-02-04,1973-01-02,"HERAT, AF"
...,...,...,...,...,...,...,...,...,...
562,0.4562,641.0,METERS,WZ004455110,-26.533,31.300,2020-01-24,1910-02-01,"MANZINI MATSAPA AIR, WZ"
567,0.5840,986.0,METERS,ZA000067743,-17.817,25.817,2020-01-04,1950-01-01,"LIVINGSTONE, ZA"
572,0.8179,1480.0,METERS,ZI000067775,-17.917,31.133,2020-02-04,1956-07-01,"HARARE KUTSAGA, ZI"
585,0.8441,1095.0,METERS,ZI000067975,-20.067,30.867,2020-01-27,1951-07-01,"MASVINGO, ZI"


In [53]:
stations = pd.read_csv('ghcnd-stations (1).txt', delimiter=r"\s+", usecols=[0, 1, 2, 3, 4, 5], engine = "python", header=None)
stations.columns = ['id', 'lat', 'long', '1', 'state', '2']
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
stations

,id,lat,long,1,state,2
0,ACW00011604,17.1167,-61.7833,10.1,ST,JOHNS
1,ACW00011647,17.1333,-61.7833,19.2,ST,JOHNS
2,AE000041196,25.3330,55.5170,34.0,SHARJAH,INTER.
3,AEM00041194,25.2550,55.3640,10.4,DUBAI,INTL
4,AEM00041217,24.4330,54.6510,26.8,ABU,DHABI
...,...,...,...,...,...,...
114340,VQC00678621,17.7500,-64.6000,9.1,VI,TAGUE
114341,VQC00679222,18.1500,-64.9500,30.5,VI,WATER
114342,VQC00679450,18.3503,-64.9167,196.6,VI,WINTBERG
114343,VQW00011624,17.7028,-64.8056,18.6,VI,CHRISTIANSTED


In [54]:
# Show stations IDs that start with U.S. 
stations = stations[stations.id.str.startswith(('US'))]

In [55]:
# Only keep temperature stations 
stations = stations.loc[stations['id'].isin(select_dates.id)]
stations

,id,lat,long,1,state,2
87690,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON
87696,USC00010160,32.9453,-85.9481,195.1,AL,ALEXANDER
87698,USC00010178,33.1272,-88.1550,59.4,AL,ALICEVILLE
87700,USC00010252,31.3072,-86.5225,76.2,AL,ANDALUSIA
87701,USC00010260,34.9628,-87.3719,231.6,AL,LEXINGTON
...,...,...,...,...,...,...
114292,USW00094993,45.6689,-96.9914,353.9,SD,SISSETON
114293,USW00094994,43.1561,-90.6775,204.8,WI,BOSCOBEL
114294,USW00094995,40.8483,-96.5650,362.4,NE,LINCOLN
114295,USW00094996,40.6953,-96.8542,418.2,NE,LINCOLN


In [56]:
# Remove lat long that are one degree apart 
stations['lat'] = stations['lat'].round()
stations['long'] = stations['long'].round()
stations = stations.groupby(['lat', 'long']).first().reset_index()
stations

,lat,long,id,1,state,2
0,19.0,-156.0,USC00513977,451.1,HI,KEALAKEKUA
1,19.0,-155.0,USC00517421,256.0,HI,PAHALA
2,20.0,-156.0,USC00511339,176.8,HI,HAWI
3,20.0,-155.0,USC00511856,329.2,HI,HONOKAA
4,21.0,-158.0,USC00510055,74.1,HI,AHUIMANU
...,...,...,...,...,...,...
1073,70.0,-149.0,USS0048V01S,9.1,AK,Prudhoe
1074,70.0,-148.0,USW00027406,18.6,AK,DEADHORSE
1075,70.0,-144.0,USW00027401,11.9,AK,BARTER
1076,71.0,-160.0,USW00027503,9.1,AK,WAINWRIGHT


In [57]:
# remove all hawaii and alaska states
stations = stations[stations.state != 'HI']
stations = stations[stations.state != 'AK']
stations.groupby('state').count()

,lat,long,id,1,2
state,,,,,
AL,19,19,19,19,19
AR,18,18,18,18,18
AZ,37,37,37,37,37
CA,53,53,53,53,53
CO,38,38,38,38,38
CT,5,5,5,5,5
DE,3,3,3,3,3
FL,27,27,27,27,27
GA,16,16,16,16,16


Use the API to get weather data

In [58]:
# Remove states that have already been processed
stations = stations[stations.state != 'AL']
stations = stations[stations.state != 'FL']
stations = stations[stations.state != 'LA']
stations = stations[stations.state != 'MS']
stations = stations[stations.state != 'TX']
stations = stations[stations.state != 'WV']
stations = stations[stations.state != 'AR']
stations = stations[stations.state != 'CA']
stations = stations[stations.state != 'AZ']
stations = stations[stations.state != 'GA']
stations = stations[stations.state != 'NC']
stations = stations[stations.state != 'NM']
stations = stations[stations.state != 'NV']
stations = stations[stations.state != 'OK']
stations = stations[stations.state != 'SC']
stations = stations[stations.state != 'TN']
stations = stations[stations.state != 'UT']
stations = stations[stations.state != 'IL']
stations = stations[stations.state != 'IN']
stations = stations[stations.state != 'KS']
stations = stations[stations.state != 'KY']
stations = stations[stations.state != 'MO']
stations = stations[stations.state != 'VA']
stations = stations[stations.state != 'CT']
stations = stations[stations.state != 'DE']
stations = stations[stations.state != 'IA']
stations = stations[stations.state != 'ID']
stations = stations[stations.state != 'MA']
stations = stations[stations.state != 'ME']
stations = stations[stations.state != 'MI']
stations = stations[stations.state != 'NE']
stations = stations[stations.state != 'NJ']
stations = stations[stations.state != 'NY']
stations = stations[stations.state != 'OH']
stations = stations[stations.state != 'OR']
stations = stations[stations.state != 'PA']
stations = stations[stations.state != 'SD']
stations = stations[stations.state != 'WI']
stations = stations[stations.state != 'WY']
stations = stations[stations.state != 'MT']
stations = stations[stations.state != 'ND']
stations = stations[stations.state != 'NH']
stations = stations[stations.state != 'VT']
stations = stations[stations.state != 'WA']

# This one isn't done!! get an error at USC00055970
#stations = stations[stations.state != 'CO']

# This one isn't done!! get an error at USC00181790
#stations = stations[stations.state != 'MD']

# This one isn't done!! get an error at USC00210643, 2016
stations = stations[stations.state != 'MN'] 


In [59]:
for state in stations.state.unique():
    print(state)

    curr_state = stations[stations.state == state]
    for id_value in curr_state.id: 
        print(id_value)

CO
USC00052441
USC00056258
USC00050128
USC00058436
USC00058429
USC00051268
USC00058793
USC00055970
USC00051609
USC00050797
USC00051964
USC00051294
USC00051539
USC00052446
USC00054076
USC00051772
USC00050214
USC00050372
USC00050263
USC00050454
USC00052668
USC00052932
USC00050306
USC00055420
USC00051932
USC00053359
USC00051186
USC00050848
USC00051179
USC00050109
USC00054242
USC00055446
USR0000CDRY
USC00053446
USC00051060
USC00050945
USC00051996
USC00054082
MD
USC00185865
USC00187330
USC00188380
USC00186620
USC00188207
USC00180700
USC00182282
USC00181790
USC00182906


In [60]:
for state in stations.state.unique():
    print(state)

    curr_state = stations[stations.state == state]
    state_df = pd.DataFrame()
    for id_value in curr_state.id: 
        max_dates_temp = []
        min_dates_temp = []
        max_temps = []
        min_temps = []
        print(id_value)

        #for each year from 2015-2019 ...
        for year in range(2012, 2018):
            year = str(year)
            print('working on year '+year)

            # API call for max temps
            #r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:USC00210643&datatypeid=TMAX&limit=1000&startdate=2016-01-01&enddate=2016-12-31', headers={'token':Token})
            r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:'+id_value+'&datatypeid=TMAX&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
            d = json.loads(r.text)

            if d!={}:
                max_temps_item = [item for item in d['results'] if item['datatype']=='TMAX']
                max_dates_temp += [item['date'] for item in max_temps_item]
                max_temps += [item['value'] for item in max_temps_item]

            # API call for min temps
            r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&stationid=GHCND:'+id_value+'&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
            d = json.loads(r.text)
            if d!={}:
                min_temps_item = [item for item in d['results'] if item['datatype']=='TMIN']
                min_dates_temp += [item['date'] for item in min_temps_item]
                min_temps += [item['value'] for item in min_temps_item]

        df_max = pd.DataFrame()
        df_max['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in max_dates_temp]
        df_max['maxTemp'] = [float(v)/10.0*1.8 + 32 for v in max_temps]
        df_max['id'] = id_value
        df_min = pd.DataFrame()
        df_min['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in min_dates_temp]
        df_min['minTemp'] = [float(v)/10.0*1.8 + 32 for v in min_temps]

        state_df = pd.concat([state_df, pd.merge(df_max, df_min, on='date', how='outer')])
    state_df.groupby(['date']).median().reset_index()

    state_df.to_csv(state + '.csv')

CO
USC00052441
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00056258
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017


/home/amy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



USC00050128
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00058436
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00058429
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00051268
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00058793
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00055970
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00051609
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
USC00050797
working on year 2012
w

In [2]:
# Find temperature by region 
states = {'FRCC': ['fl'], 
                  'WECC': ['ca','or','wa', 'nv','mt','id','wy','ut','co','az','nm','tx'],
                  'SPP' : ['nm','ks','tx','ok','la','ar','mo'],
                  'RFC' : ['wi','mi','il','in','oh','ky','wv','va','md','pa','nj'],
                  'NPCC' : ['ny','ct','de','ri','ma','vt','nh','me'],
                  'SERC' : ['mo','ar','tx','la','ms','tn','ky','il','va','al','fl','ga','sc','nc'],
                  'MRO': ['ia','il','mi','mn','mo','mt','nd','ne','sd','wi','wy'], 
                  'TRE': ['ok','tx']}

In [33]:
for region in states:
    print("Calculating for " + region)
    region_df = pd.DataFrame()
    for s in states[region]:
        try:
            df1 = pd.read_csv("results/" + s.upper() + ".csv")
            print("processing " + s)
            df1 = df1.groupby("date").median()
            df1 = df1[['maxTemp','minTemp']]
            
            # combine with what the file contains
            region_df = pd.concat((region_df, df1))
            region_df.groupby(region_df.index).median()
        except IOError:
            print(s)
    
    # save as csv 
    region_df.to_csv("nerc_results/" + region + ".csv")

Calculating for FRCC
processing fl
Calculating for WECC
processing ca
processing or
processing wa
processing nv
processing mt
processing id
processing wy
processing ut
processing co
processing az
processing nm
processing tx
Calculating for SPP
processing nm
processing ks
processing tx
processing ok
processing la
processing ar
processing mo
Calculating for RFC
processing wi
processing mi
processing il
processing in
processing oh
processing ky
processing wv
processing va
processing md
processing pa
processing nj
Calculating for NPCC
processing ny
processing ct
processing de
ri
processing ma
processing vt
processing nh
processing me
Calculating for SERC
processing mo
processing ar
processing tx
processing la
processing ms
processing tn
processing ky
processing il
processing va
processing al
processing fl
processing ga
processing sc
processing nc
Calculating for MRO
processing ia
processing il
processing mi
processing mn
processing mo
processing mt
processing nd
processing ne
processing sd

In [22]:
df1 = pd.read_csv("results/AL.csv")
df2 = pd.read_csv("results/AR.csv")

df1 = df1.groupby("date").median()
df1 = df1[['maxTemp','minTemp']]

df2 = df2.groupby("date").median()
df2 = df2[['maxTemp','minTemp']]

df3 = pd.concat((df1, df2))
df3.groupby(df3.index).median()


,maxTemp,minTemp
date,,
2012-01-01,66.290,38.525
2012-01-02,64.040,34.520
2012-01-03,48.245,23.540
2012-01-04,44.510,20.930
2012-01-05,57.020,25.970
...,...,...
2017-12-27,45.500,26.060
2017-12-28,38.030,23.000
2017-12-29,38.480,23.450
